In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# imports
import pandas as pd
import itertools
import os
from tqdm import tqdm
import time

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()
from auctions.webutils import *
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from open_spiel.python.examples.ubc_dispatch import verify_config, get_cluster_details, write_and_submit, dispatch_eval_database, dispatch_single_br_database

plt.style.use('https://raw.githubusercontent.com/gregdeon/plots/main/style.mplstyle')
plt.rcParams['axes.titlesize'] = 7
plt.rcParams['figure.dpi'] = 300

In [23]:
# Step 1: Find a game that you want to do this on. 
# (Ensure you have a the rho=0 game)

experiment = 'feb8_3p_5supply_v2'
run_name = 'feb8_3p_5supply_feb8_3p_5supply_3_base_dev1000_rho1_t3-cfr_10external_plus_linear-100'
run_name_tb = 'feb8_3p_5supply_feb8_3p_5supply_3_base_dev1000_rho1_t3_tie_break-cfr_10external_plus_linear-100'

run = EquilibriumSolverRun.objects.get(name=run_name, experiment__name=experiment)
run_tb = EquilibriumSolverRun.objects.get(name=run_name_tb, experiment__name=experiment)

In [27]:
Experiment.objects.get(name='feb8_3p_5supply_v2').equilibriumsolverrun_set.all()

100

In [24]:
print(run.equilibriumsolverruncheckpoint_set.last().get_modal_eval().nash_conv)
print(run.equilibriumsolverruncheckpoint_set.last().get_modal_eval().heuristic_conv)
print(run_tb.equilibriumsolverruncheckpoint_set.last().get_modal_eval().nash_conv)
print(run_tb.equilibriumsolverruncheckpoint_set.last().get_modal_eval().heuristic_conv)

0.0
0.0
None
0.0


In [5]:
# Step 2: Code that will run HC and NC for longer 
BASE_OUTPUT_DIR = os.environ['CLOCK_AUCTION_OUTPUT_ROOT']
BR_DIR = 'best_responses'

def dispatch_one_off_command(cmd, experiment_name, run_name, extra_name='', cluster='ada_cpu_short', submit=True):
    spiel_path, config_dir, pydir, manage_path = verify_config()
    cluster_details = get_cluster_details(cluster)
    command = f'python {manage_path} {cmd}'

    slurm_job_name = f'one_off_{experiment_name}_{run_name}_{extra_name}'
    job_file_text = f"""{cluster_details['shell']}
{cluster_details['preamble']}
#SBATCH --job-name={slurm_job_name}
#SBATCH --time=0-3:00:00 # days-hh:mm:ss
#SBATCH -e slurm-%j-{slurm_job_name}.err
#SBATCH -o slurm-%j-{slurm_job_name}.out

{cluster_details['load_py']}
export OPENSPIEL_PATH={spiel_path}

CMD=`{command}`
echo $CMD
eval $CMD
"""

    experiment_dir = f'{BASE_OUTPUT_DIR}/{experiment_name}/{run_name}/{BR_DIR}'
    write_and_submit(experiment_dir, slurm_job_name, job_file_text, submit)

    logging.info(f"Dispatched experiment!")

for r in [run, run_tb]:
    for heuristic in [True, False]:
        cp = r.equilibriumsolverruncheckpoint_set.last()
        cmd = f"nashconv --time_limit_seconds 3600 --t {cp.t} --run_name {cp.equilibrium_solver_run.name} --experiment_name {cp.equilibrium_solver_run.experiment.name} --heuristic {heuristic}"
        dispatch_one_off_command(cmd, cp.equilibrium_solver_run.experiment.name, cp.equilibrium_solver_run.name, extra_name='heuristic' if heuristic else 'nc')

Submitted batch job 1090716
Submitted batch job 1090717
Submitted batch job 1090718
Submitted batch job 1090719


In [28]:
#### ALL RUNS ###
for r in Experiment.objects.get(name='feb8_3p_5supply_v2').equilibriumsolverrun_set.all():
    for heuristic in [True, False]:
        cp = r.equilibriumsolverruncheckpoint_set.last()
        cmd = f"nashconv --time_limit_seconds 3600 --t {cp.t} --run_name {cp.equilibrium_solver_run.name} --experiment_name {cp.equilibrium_solver_run.experiment.name} --heuristic {heuristic}"
        dispatch_one_off_command(cmd, cp.equilibrium_solver_run.experiment.name, cp.equilibrium_solver_run.name, extra_name='heuristic' if heuristic else 'nc')



Submitted batch job 1090755
Submitted batch job 1090756
Submitted batch job 1090757
Submitted batch job 1090758
Submitted batch job 1090759
Submitted batch job 1090760
Submitted batch job 1090761
Submitted batch job 1090762
Submitted batch job 1090763
Submitted batch job 1090764
Submitted batch job 1090765
Submitted batch job 1090766
Submitted batch job 1090767
Submitted batch job 1090768
Submitted batch job 1090769
Submitted batch job 1090770
Submitted batch job 1090771
Submitted batch job 1090772
Submitted batch job 1090773
Submitted batch job 1090774
Submitted batch job 1090775
Submitted batch job 1090776
Submitted batch job 1090777
Submitted batch job 1090778
Submitted batch job 1090779
Submitted batch job 1090780
Submitted batch job 1090781
Submitted batch job 1090782
Submitted batch job 1090783
Submitted batch job 1090784
Submitted batch job 1090785
Submitted batch job 1090786
Submitted batch job 1090787
Submitted batch job 1090788
Submitted batch job 1090789
Submitted batch job 

In [21]:
# Step 4: Code that will run PPO-BR 
config = 'ppo_jun8_23/ppo_76'

for r in [run, run_tb]:
    game = r.game.load_as_spiel()
    for br_player in range(game.num_players()):
        cp = r.equilibriumsolverruncheckpoint_set.last()
        experiment_name = cp.equilibrium_solver_run.experiment.name
        run_name = cp.equilibrium_solver_run.name
        t = cp.t
        dispatch_single_br_database(experiment_name, run_name, t, br_player, config, True, '--total_timesteps 500', django_command='ppo_br', cluster='ada_cpu_short')

Submitted batch job 1090749
Submitted batch job 1090750
Submitted batch job 1090751
Submitted batch job 1090752
Submitted batch job 1090753
Submitted batch job 1090754


In [ ]:
# Step 5: Code that will run single agent MCCFR
# Require a game wrapper

In [30]:
!squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "one_off"  | wc -l

178


In [ ]:
!squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "one_off" | awk '{print $1}' | xargs scancel

In [19]:
!squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "ppo_br" 

In [25]:
!sinfo

PARTITION     AVAIL  TIMELIMIT  NODES  STATE NODELIST 
archipelago*     up   infinite     14   idle archipelago[15-28] 
ada_cpu_long     up   infinite     16    mix ada[01-16] 
ada_cpu_short    up    3:00:00     19    mix ada[01-17,20-21] 
ada_cpu_short    up    3:00:00      2  alloc ada[18-19] 
ada_gpu          up   infinite      3    mix ada[17,20-21] 
ada_gpu          up   infinite      2  alloc ada[18-19] 
plai             up   infinite      1  down* plai10 
plai             up   infinite      1   drng plai11 
plai             up   infinite      1    mix plai12 
plai             up   infinite      9   idle plai[01-09] 
plai_towers      up   infinite      3   idle alexandria,berkeley,lowpoint 
blackboxml       up   infinite      1   idle chicago 
plai_cpus        up   infinite      2    mix nodeb,noded 
plai_cpus        up   infinite      2   idle nodea,nodec 
edith            up   infinite      4    mix edith[1-2,4],friday 
edith            up   infinite      1  alloc edith3 
edith